In [1]:
from Loader import fileloader as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
import pandas as pd
import numpy as np
from datetime import datetime
import time
import posixpath

month = '201906'
month_unitario = 201812

inifile = fl.ReadIniFile(mercado="empresas")

defaultpath = inifile.getDataPath()
testpath = inifile.getTestPath()

#testpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/'
#defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/'

#defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/meses previos/201809/'

#*****************Actualizar periodo*************
#defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/meses previos/201804/'


INFO:    Values loaded from globals.ini:
INFO:     * mainpath C:/Users/jsaenza/OneDrive - Entel Peru S.A
INFO:     * scriptspath C:/Users/jsaenza/Documents/Comisiones/Comisiones-python
INFO:     * databasepath C:/Users/jsaenza/OneDrive - Entel Peru S.A/SQLiteDB
INFO:    datapath value is C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test


In [2]:
# Configurando los archivos de configuración para importar archivos tipo : xlsx, txt, ini
parser = inifile.getIniFileParser()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [3]:
# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
#inidb = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/mydbconfig.ini')
#dbparser = inidb.readFile()

dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)


INFO:    Setting database path to C:/Users/jsaenza/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


In [5]:
# Carga de Panel de Plataformas Comerciales: 

# Preparación de Data : Quitar formulas y quitar pestañas ocultas. Si hay intercambio de supervisores (o nuevo supervisor) 
# actualizar logins equivalentes y metricas conjuntas
# File : _Plataformas Comerciales Panel

logins = loader.loadFile('Logins')
metricasconjuntas = loader.loadFile('Metricas_conjuntas')
kpispltfrs = loader.loadFile('Kpis_plataformas')
comisionantespltfrs = loader.loadFile('Comisionantes_plataformas')
panelpltfrs0 = loader.loadHistoricalFile('Panel_plataformas')
periodo = loader.getPeriodo()

paramspltfrs= {'tipo' : 'plataformas', 'periodo' : periodo.upper(), 'keycol': 'NOMBRES', 'logins' : logins, 
               'metricasconjuntas' : metricasconjuntas, 'kpis' : kpispltfrs, 'comisionantes' : comisionantespltfrs, 
               'cuotas' : panelpltfrs0, 'resultados' : None}

panelpltfrs = loader.prepareHistoricalFiles(paramspltfrs)

panelpltfrs.to_csv(posixpath.join(testpath, month + '_panelpltfrs.csv'),encoding='utf-8-sig')

#"""

INFO:    Values loaded from globals.ini:
INFO:    Values loaded from globals.ini:
INFO:    Values loaded from globals.ini:


Archivo: C:/Users/jsaenza/Documents/Comisiones/Comisiones-python/Data/logins equivalentes.xlsx
Hoja Importada: Similar
El tamaño de Logins es 146 registros
Archivo: C:/Users/jsaenza/Documents/Comisiones/Comisiones-python/Data/metricas conjuntas.xlsx
Hoja Importada: reporte
El tamaño de Metricas_conjuntas es 87 registros
Archivo: C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Comisiones Plataformas Comerciales/201906_Comisiones Plataformas Comerciales.xlsm


INFO:    Values loaded from globals.ini:


Hoja Importada: Leyenda
El tamaño de Kpis_plataformas es 192 registros
El tamaño de Comisionantes_plataformas es 129 registros
Archivo: C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx\201906_Plataformas Comerciales Panel Presenciales.xlsx
Hoja Importada: Canales Presenciales
Hoja Importada: Plataforma Outbound
Archivo: C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx\201906_Plataformas Comerciales Panel_cerrado2.xlsx
Hoja Importada: Resumen Indicadores
Hoja Importada: Plataforma Carterizada
Hoja Importada: Plataforma Inbound
El tamaño de Panel_plataformas es 6060 registros


In [4]:
#Carga de Panel de voz
#*************************
# Agregar mes enero faltante en cuotas de GC
# Asegúrate en colocar 0 en la cabecera de los resultados y eliminar toda la data fuera del formato

#"""
logins = loader.loadFile('Logins') # ojo con tener logins duplicados. Sale errores raros
metricasconjuntas = loader.loadFile('Metricas_conjuntas')
kpisvoz = loader.loadFile('Kpis_voz_gc')
comisionantesvoz = loader.loadFile('Comisionantes_voz_gc')
cuotasvoz = loader.loadHistoricalFile('Cuotas')
#"""                                                                                                                                               

periodo = loader.getPeriodo()

#"""

# Preparar los resultados antes de enviar el cerrado
# GCE_Resultados debe tener dos o mas Hojas sino lanza error

#resultadosvoz = loader.loadHistoricalFile('Resultados') # Por precaución eliminar tres columnas posteriores a dic-18.
resultadosvoz = pd.DataFrame() # Usar si no hay resultados
                                                         
#churn = loader.loadHistoricalFile('Churn')
#calidad = loader.loadHistoricalFile('Calidad')

#qnp = loader.loadHistoricalFile('QNP') 
#tracking = loader.loadHistoricalFile('Tracking') # Asegurarse el nombre del kpi en resultados
#epa = loader.loadHistoricalFile('EPA') # Asegurarse el nombre del kpi en resultados

#resultadosvoz = resultadosvoz.append(churn, ignore_index = True)
#resultadosvoz = resultadosvoz.append(calidad, ignore_index = True)

#resultadosvoz = resultadosvoz.append(qnp, ignore_index = True)
#resultadosvoz = resultadosvoz.append(tracking, ignore_index = True)
#resultadosvoz = resultadosvoz.append(epa, ignore_index = True)

#"""

INFO:    Values loaded from globals.ini:
INFO:    Values loaded from globals.ini:
INFO:    Values loaded from globals.ini:


Archivo: C:/Users/jsaenza/Documents/Comisiones/Comisiones-python/Data/logins equivalentes.xlsx
Hoja Importada: Similar
El tamaño de Logins es 146 registros
Archivo: C:/Users/jsaenza/Documents/Comisiones/Comisiones-python/Data/metricas conjuntas.xlsx
Hoja Importada: reporte
El tamaño de Metricas_conjuntas es 87 registros
Archivo: C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Comisiones Grandes Cuentas/201906_Comisiones Grandes Cuentas.xlsm


INFO:    Values loaded from globals.ini:


Hoja Importada: Leyenda
El tamaño de Kpis_voz_gc es 133 registros
El tamaño de Comisionantes_voz_gc es 210 registros
Archivo: C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx\201906_Formato de cuotas.xlsx
Hoja Importada: DDNN
Hoja Importada: CORPORACIONES
Hoja Importada: VD PYME
Hoja Importada: DEALERS LIMA
Hoja Importada: DEALER REGIONES
Hoja Importada: VENTA REGIONAL EMPRESA
Hoja Importada: KAM REGIONAL
Hoja Importada: GRANDES CLIENTES
Hoja Importada: DATA
Hoja Importada: CHURN
El tamaño de Cuotas es 1366 registros


In [5]:
# Preparando la ultima data : HC, VAS

#hc = loader.loadFile('HC')
#vas = dbmanager.loadData('View_VAS_Voz')
#vas = pd.DataFrame() # temporal mientras se arregla el código

#periodo = loader.getPeriodo() # arriba se repite


"""
criteria = {'VAS' : {'colgroupby' : ['GERENCIA2' , 'ZONAVENTA', 'DEPARTAMENTO', 'VENDEDOR_CROSS_SELLING'], 
                     'colsum' : 'ACCESS_REAL'},
            'HC' : {'colfilter' : 'CARGO', 'colfilteritem' : 'CONSULTOR', 'colgroupby' : 'DATOS', 'colsum' : 'VENDEDORES'}}

otherparams = {'periodo' : periodo.upper(), 'frames' : {'HC' : hc, 'VAS' : vas}, 'criteria': criteria}
otherdfs = loader.prepareOtherPlainFiles(otherparams) 


for df in otherdfs.keys():
    resultadosvoz = resultadosvoz.append(otherdfs[df], ignore_index = True)

"""


#"""
paramsvoz = {'tipo' : 'voz', 'periodo' : periodo.upper(), 'keycol': 'LOGIN', 'logins' : logins, 
             'metricasconjuntas' : metricasconjuntas, 'kpis' : kpisvoz, 'comisionantes' : comisionantesvoz, 
             'cuotas' : cuotasvoz, 'resultados' : resultadosvoz}


panelvoz = loader.prepareHistoricalFiles(paramsvoz)          
panelvoz.to_csv(posixpath.join(testpath, month + '_panelvoz_gc.csv'),encoding='utf-8-sig')
#"""

C:\Users\jsaenza\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [6]:
cuotasvoz.to_csv(testpath + 'cuotas_brutas.csv')

In [5]:
testpath

'C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test'

In [6]:
cuotasvoz

,DATOS,ABR-19
0,Ventas S/,NaN
1,D.LOPEZ,NaN
2,DCANALE,21305.00
3,JALVAREZ,21305.00
4,EJECUTIVO DE DESARROLLO PYME,1850.00
5,EJECUTIVO DE DESARROLLO PYME SENIOR,2350.00
6,Ventas,NaN
7,D.LOPEZ,NaN
8,DCANALE,480.00
9,JALVAREZ,480.00
